# Comparison of cost vs quality for 50 random cities
### Eric P. Hanson

We will use `TravelingSalesmanExact` to compute the exact cost and compare to the estimated best costs found by `TravelingSalesmanHeuristics` with various settings of `quality`.
First we load the packages:

In [ ]:
using TravelingSalesmanExact, GLPK
using TravelingSalesmanHeuristics
using TravelingSalesmanExact: ATT, euclidean_distance
using TravelingSalesmanBenchmarks
using Plots
gr(fmt=:svg)
repo_directory = TravelingSalesmanBenchmarks.repo_directory;
data_directory = joinpath(repo_directory, "data");

Then we generate a cost matrix by choosing cities at random:

In [ ]:
N = 50
cities = [100*rand(2) for _ = 1:N]
cost = [ euclidean_distance(cities[i], cities[j]) for i=1:N, j=1:N ]

Now we will compute an optimal tour and cost, and plot these versus those found by the `tsp_solve` function of `TravelingSalesmanHeuristics`.

In [ ]:
t_exact, c_exact = get_optimal_tour(cost, with_optimizer(GLPK.Optimizer))

c(q) = solve_tsp(cost; quality_factor = q)[2]

qs = range(10, stop = 100, step = 10)

plot(qs, c, xlabel="quality", ylabel="Cost", label="solve_tsp 1", title="random cities")
for j = 2:5
    plot!(qs, c, label="solve_tsp $j")
end
hline!([c_exact], label="Exact cost")

We've run `tsp_solve` five times for each quality, since the cost will vary from run to run due to the randomness of the heuristics.

We can compare plots of the tours; we will override `TravelingSalesmanExact` to use `GR` for better plotting, since we load it anyway.

In [ ]:
function TravelingSalesmanExact.plot_cities(cities)
    n = length(cities)
    inc(a) = a == n ? one(a) : a + 1
    Plots.plot([cities[inc(j)][1] for j = 0:n], [cities[inc(j)][2] for j = 0:n])
end

Now, an optimal tour:

In [ ]:
plot_cities(cities[t_exact])

A heuristically-derived tour with `quality_factor` of `100`:

In [ ]:
t = solve_tsp(cost; quality_factor = 100)[1]
plot_cities(cities[t])

In [ ]:
TravelingSalesmanBenchmarks.bench_footer(WEAVE_ARGS[:file])